In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import statsmodels.api as sm
from statsmodels.tools.eval_measures import mse, rmse

%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

import warnings
warnings.filterwarnings(action="ignore")

- Reimplement your model from the previous lesson.

In [2]:
df = pd.read_csv("https://djl-lms-assets.s3.eu-central-1.amazonaws.com/datasets/house_prices.csv", sep = ";")
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.000,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.000,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.000,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.000,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.000,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
numerical = ['MSSubClass', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'MiscVal', 'MoSold', 'YrSold', 'SalePrice']
categorical = ['Alley', 'BldgType', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'BsmtFinType2', 'BsmtQual', 'CentralAir', 'Condition1', 'Condition2',
       'Electrical', 'ExterCond', 'ExterQual', 'Exterior1st', 'Exterior2nd',
       'Fence', 'FireplaceQu', 'Foundation', 'Functional', 'GarageCond',
       'GarageFinish', 'GarageQual', 'GarageType', 'Heating', 'HeatingQC',
       'HouseStyle', 'KitchenQual', 'LandContour', 'LandSlope', 'LotConfig',
       'LotShape', 'MSZoning', 'MasVnrType', 'MiscFeature', 'Neighborhood',
       'PavedDrive', 'PoolQC', 'RoofMatl', 'RoofStyle', 'SaleCondition',
       'SaleType', 'Street', 'Utilities']

In [4]:
num = 0
for i in numerical:
    df[i].fillna(df[i].mean,inplace=True)
for i in categorical:
    df[i].fillna(df[i].mode()[0],inplace=True)
num = 0
for i in df.isnull().sum():
    if i !=0:
        num+=1
if num == 0:
    print("There is no null value")
else:
    print(f"There is {num} pieces null value")
df.dropna(axis=0,inplace=True)

There is 3 pieces null value


In [5]:
for i in categorical:
    dummy = pd.get_dummies(df[i],drop_first=True,prefix=i,prefix_sep='_')
    df = pd.concat([df,dummy],axis=1)
df.drop(columns=categorical,inplace=True)
df.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleCondition_Partial,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,Street_Pave
0,1,60,65.000,8450,7,5,2003,2003,196.000,706,...,0,0,0,0,0,0,0,0,1,1
1,2,20,80.000,9600,6,8,1976,1976,0.000,978,...,0,0,0,0,0,0,0,0,1,1
2,3,60,68.000,11250,7,5,2001,2002,162.000,486,...,0,0,0,0,0,0,0,0,1,1
3,4,70,60.000,9550,7,5,1915,1970,0.000,216,...,0,0,0,0,0,0,0,0,1,1
4,5,60,84.000,14260,8,5,2000,2000,350.000,655,...,0,0,0,0,0,0,0,0,1,1


In [6]:
Y = df["SalePrice"]
X = df.drop(["SalePrice","Id"],axis=1)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 465)

In [8]:
X_train = sm.add_constant(X_train)

sonuclar = sm.OLS(y_train, X_train).fit()

sonuclar.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.947
Model:                            OLS   Adj. R-squared:                  0.929
Method:                 Least Squares   F-statistic:                     54.08
Date:                Sat, 10 Jul 2021   Prob (F-statistic):               0.00
Time:                        10:49:33   Log-Likelihood:                -10109.
No. Observations:                 896   AIC:                         2.066e+04
Df Residuals:                     673   BIC:                         2.173e+04
Df Model:                         222                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -3.172e+05   9.27e+05     -0.342      0.732   -2.14e+06     1.5e+06
MSSubClass              -29.3294    129.828     -0.226      0.821    -284.245     225.587
LotFrontage              93.4909     63.863      1.464      0.144     -31.905     218.886
LotArea                   0.9722      0.161      6.040      0.000       0.656       1.288
OverallQual            6645.4930   1440.561      4.613      0.000    3816.958    9474.028
OverallCond            5795.8927   1241.482      4.669      0.000    3358.249    8233.536
YearBuilt               257.5328    107.068      2.405      0.016      47.306     467.760
YearRemodAdd             45.0105     75.587      0.595      0.552    -103.405     193.425
MasVnrArea               19.2681      7.171      2.687      0.007       5.188      33.348
BsmtFinSF1               19.4428      3.726      5.218      0.000      12.126      26.759
BsmtFinSF2                4.6223      8.072      0.573      0.567     -11.227      20.471
BsmtUnfSF                -1.5110      3.614     -0.418      0.676      -8.608       5.586
TotalBsmtSF              22.5542      4.860      4.641      0.000      13.012      32.096
1stFlrSF                 17.9021      9.770      1.832      0.067      -1.281      37.085
2ndFlrSF                 31.6732      8.089      3.915      0.000      15.790      47.557
LowQualFinSF            -12.2330     23.117     -0.529      0.597     -57.623      33.157
GrLivArea                37.3423      8.997      4.150      0.000      19.676      55.009
BsmtFullBath          -1543.5627   2686.132     -0.575      0.566   -6817.771    3730.645
BsmtHalfBath          -1442.9714   3918.584     -0.368      0.713   -9137.092    6251.149
FullBath               7924.7056   3043.468      2.604      0.009    1948.870    1.39e+04
HalfBath               4391.0067   2818.017      1.558      0.120   -1142.157    9924.170
BedroomAbvGr          -4993.7040   1876.991     -2.660      0.008   -8679.167   -1308.241
KitchenAbvGr          -1.702e+04   9774.014     -1.742      0.082   -3.62e+04    2169.478
TotRmsAbvGrd           1104.5896   1246.306      0.886      0.376   -1342.525    3551.705
Fireplaces              652.9286   1995.326      0.327      0.744   -3264.885    4570.742
GarageYrBlt             -30.3829     77.807     -0.390      0.696    -183.156     122.390
GarageCars             4909.7858   2853.224      1.721      0.086    -692.506    1.05e+04
GarageArea                9.7548      9.890      0.986      0.324      -9.664      29.173
WoodDeckSF               12.2958      7.875      1.561      0.119      -3.166      27.758
OpenPorchSF              14.5288     16.599      0.875      0.382     -18.063      47.120
EnclosedPorch            14.0564     18.080      0.777      0.437     -21.444      49.556
3SsnPorch              

In [9]:
X_test = sm.add_constant(X_test)
y_preds = sonuclar.predict(X_test)

In [10]:
print("Ortalama Mutlak Hata (MSE)        : {}".format(mean_absolute_error(y_test, y_preds)))
print("Ortalama Kare Hata (MSE)          : {}".format(mse(y_test, y_preds)))
print("Kök Ortalama Kare Hata (RMSE)     : {}".format(rmse(y_test, y_preds)))
print("Ortalama Mutlak Yüzde Hata (MAPE) : {}".format(np.mean(np.abs((y_test - y_preds) / y_test)) * 100))

Ortalama Mutlak Hata (MSE)        : 19573.691118411065
Ortalama Kare Hata (MSE)          : 1770364311.1594722
Kök Ortalama Kare Hata (RMSE)     : 42075.6973936199
Ortalama Mutlak Yüzde Hata (MAPE) : 11.322690893315409


In [11]:
X = df[["LotFrontage","LotArea","OverallQual","OverallCond","YearBuilt","MasVnrArea","BsmtFinSF1","TotalBsmtSF","1stFlrSF","2ndFlrSF","GrLivArea","FullBath","HalfBath","BedroomAbvGr","KitchenAbvGr","ScreenPorch","PoolArea","Alley_Pave","BsmtExposure_Gd","BsmtExposure_Mn","BsmtExposure_No"]]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 465)

X_train = sm.add_constant(X_train)

sonuclar = sm.OLS(y_train, X_train).fit()

sonuclar.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.810
Model:                            OLS   Adj. R-squared:                  0.806
Method:                 Least Squares   F-statistic:                     178.0
Date:                Sat, 10 Jul 2021   Prob (F-statistic):          3.63e-298
Time:                        10:49:33   Log-Likelihood:                -10679.
No. Observations:                 896   AIC:                         2.140e+04
Df Residuals:                     874   BIC:                         2.151e+04
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const           -7.821e+05    1.4e+05     -5.581      0.000   -1.06e+06   -5.07e+05
LotFrontage       -13.1366     63.774     -0.206      0.837    -138.304     112.031
LotArea             0.7300      0.165      4.423      0.000       0.406       1.054
OverallQual      2.046e+04   1586.647     12.895      0.000    1.73e+04    2.36e+04
OverallCond      5514.5692   1333.846      4.134      0.000    2896.654    8132.484
YearBuilt         366.1834     71.026      5.156      0.000     226.783     505.584
MasVnrArea         40.3295      7.764      5.195      0.000      25.092      55.567
BsmtFinSF1         10.7022      3.399      3.148      0.002       4.030      17.374
TotalBsmtSF         7.2812      5.682      1.281      0.200      -3.871      18.434
1stFlrSF           14.7695     29.562      0.500      0.617     -43.251      72.790
2ndFlrSF           -0.8397     28.947     -0.029      0.977     -57.654      55.975
GrLivArea          56.0508     28.481      1.968      0.049       0.153     111.949
FullBath         7164.4985   3853.052      1.859      0.063    -397.817    1.47e+04
HalfBath         3590.8344   3678.251      0.976      0.329   -3628.403    1.08e+04
BedroomAbvGr    -6655.4721   2072.138     -3.212      0.001   -1.07e+04   -2588.525
KitchenAbvGr    -2.448e+04   6517.280     -3.756      0.000   -3.73e+04   -1.17e+04
ScreenPorch        51.3040     22.082      2.323      0.020       7.963      94.645
PoolArea         -268.7734     38.742     -6.938      0.000    -344.811    -192.735
Alley_Pave      -1.443e+04   7351.009     -1.964      0.050   -2.89e+04      -6.693
BsmtExposure_Gd  1.558e+04   5473.973      2.846      0.005    4837.173    2.63e+04
BsmtExposure_Mn -4573.0783   5634.156     -0.812      0.417   -1.56e+04    6484.979
BsmtExposure_No -1.066e+04   3753.686     -2.840      0.005    -1.8e+04   -3294.027
==============================================================================
Omnibus:                      371.809   Durbin-Watson:                   2.036
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            36101.179
Skew:                          -0.911   Prob(JB):                         0.00
Kurtosis:                      34.043   Cond. No.                     1.54e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.54e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [12]:
X_test = sm.add_constant(X_test)
y_preds = sonuclar.predict(X_test)

print("Ortalama Mutlak Hata (MSE)        : {}".format(mean_absolute_error(y_test, y_preds)))
print("Ortalama Kare Hata (MSE)          : {}".format(mse(y_test, y_preds)))
print("Kök Ortalama Kare Hata (RMSE)     : {}".format(rmse(y_test, y_preds)))
print("Ortalama Mutlak Yüzde Hata (MAPE) : {}".format(np.mean(np.abs((y_test - y_preds) / y_test)) * 100))

Ortalama Mutlak Hata (MSE)        : 23987.26254882257
Ortalama Kare Hata (MSE)          : 2228317103.1055264
Kök Ortalama Kare Hata (RMSE)     : 47205.05378776224
Ortalama Mutlak Yüzde Hata (MAPE) : 13.224135226316507


- Try OLS, Lasso, Ridge and ElasticNet regressions using the same model specification. This time, you need to do cross-validation to choose the best hyperparameter values for your models. Which model is the best? Why?

In [13]:
from sklearn.linear_model import Ridge

# Fitting a ridge regression model. Alpha is the regularization
# parameter (usually called lambda). As alpha gets larger, parameter
# shrinkage grows more pronounced.
ridgeregr = Ridge(alpha=10**2) 
ridgeregr.fit(X_train, y_train)

# We are making predictions here
y_preds_train = ridgeregr.predict(X_train)
y_preds_test = ridgeregr.predict(X_test)

print("R-squared of the model in training set is: {}".format(ridgeregr.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(ridgeregr.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

R-squared of the model in training set is: 0.8059134204212881
-----Test set statistics-----
R-squared of the model in test set is: 0.6624384960590919
Mean absolute error of the prediction is: 23612.497204721563
Mean squared error of the prediction is: 2224586955.7949514
Root mean squared error of the prediction is: 47165.52719725447
Mean absolute percentage error of the prediction is: 12.909097224428642


In [14]:
params_ridlas = {'alpha':[10**x for x in range (1,15)]}

params_elnet = {'alpha':[10**x for x in range (1,15)],
         'l1_ratio': [x**-10 for x in range (1,10)]
         }

In [15]:
from sklearn.model_selection import GridSearchCV
grid_cv = GridSearchCV(estimator=ridgeregr,
                       param_grid = params_ridlas,
                       cv = 10
                      )
grid_cv.fit(X_train, y_train)

print("Best Parameters : ", grid_cv.best_params_)
print("Best Score      : ", grid_cv.best_score_)

Best Parameters :  {'alpha': 10}
Best Score      :  0.718477491091433


In [16]:
from sklearn.linear_model import Lasso

lassoregr = Lasso(alpha=10**2.5) 
lassoregr.fit(X_train, y_train)

# We are making predictions here
y_preds_train = lassoregr.predict(X_train)
y_preds_test = lassoregr.predict(X_test)

print("R-squared of the model in training set is: {}".format(lassoregr.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(lassoregr.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

R-squared of the model in training set is: 0.8087898940701793
-----Test set statistics-----
R-squared of the model in test set is: 0.6634471734909854
Mean absolute error of the prediction is: 23833.824892754128
Mean squared error of the prediction is: 2217939602.2567115
Root mean squared error of the prediction is: 47095.006128640765
Mean absolute percentage error of the prediction is: 13.140990092622637


In [17]:
grid_cv = GridSearchCV(estimator=lassoregr,
                       param_grid = params_ridlas,
                       cv = 10
                      )
grid_cv.fit(X_train, y_train)

print("Best Parameters : ", grid_cv.best_params_)
print("Best Score      : ", grid_cv.best_score_)

Best Parameters :  {'alpha': 10}
Best Score      :  0.7184814409523602


In [18]:
from sklearn.linear_model import ElasticNet

elasticregr = ElasticNet(alpha=10**2, l1_ratio=0.5) 
elasticregr.fit(X_train, y_train)

# We are making predictions here
y_preds_train = elasticregr.predict(X_train)
y_preds_test = elasticregr.predict(X_test)

print("R-squared of the model in training set is: {}".format(elasticregr.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(elasticregr.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

R-squared of the model in training set is: 0.7285960008784025
-----Test set statistics-----
R-squared of the model in test set is: 0.5961037828422164
Mean absolute error of the prediction is: 26909.547164205826
Mean squared error of the prediction is: 2661743847.252848
Root mean squared error of the prediction is: 51592.090937011344
Mean absolute percentage error of the prediction is: 14.811517360987056


In [19]:
grid_cv = GridSearchCV(estimator=elasticregr,
                       param_grid = params_elnet,
                       cv = 10
                      )
grid_cv.fit(X_train, y_train)

print("Best Parameters : ", grid_cv.best_params_)
print("Best Score      : ", grid_cv.best_score_)

Best Parameters :  {'alpha': 10, 'l1_ratio': 1.0}
Best Score      :  0.7184814409523602


The best scores of the three models are almost identical. Therefore, it is not easy to say that this model is superior to this model.